# 🔬 Advanced Feature Engineering for Titanic Dataset
## Sophisticated Data Science Techniques for Enhanced Predictive Power

This notebook demonstrates advanced feature engineering techniques that showcase professional data science skills:

1. **NLP Analysis on Passenger Names** - Extract ethnic, geographic, and social patterns
2. **Spatial Analysis of Cabin Positions** - Analyze survival corridors and proximity effects
3. **Interaction Effects** - Model complex relationships between social class and family dynamics
4. **Feature Selection & Validation** - Statistical rigor in feature evaluation

### Key Skills Demonstrated:
- Natural Language Processing for categorical data
- Spatial data analysis and geographic information systems
- Statistical interaction modeling
- Advanced feature selection techniques
- Domain expertise in maritime disaster analysis

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# NLP and text processing
import re
from collections import Counter, defaultdict
from itertools import combinations

# Advanced statistics and modeling
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr, spearmanr
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

# Visualization enhancements
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")
print("🚢 Advanced Feature Engineering Notebook Ready")

## 📊 Data Loading and Initial Exploration

In [ ]:
# Load the dataset
df = pd.read_csv('Titanic-Dataset.csv')
print(f"Dataset shape: {df.shape}")
print(f"Survival rate: {df['Survived'].mean():.1%}")

# Create a working copy for feature engineering
data = df.copy()

# Display basic info
display(data.head())
print("\n📋 Missing Values:")
print(data.isnull().sum()[data.isnull().sum() > 0])

# 🗣️ Section 1: NLP Analysis on Passenger Names
## Extracting Ethnic, Geographic, and Social Patterns

### 1.1 Name Component Analysis

In [ ]:
def extract_name_components(name):
    """
    Extract detailed components from passenger names
    """
    if pd.isna(name):
        return {'title': 'Unknown', 'surname': 'Unknown', 'first_name': 'Unknown', 
                'middle_name': '', 'maiden_name': '', 'nickname': ''}
    
    # Extract title (Mr., Mrs., Miss., etc.)
    title_match = re.search(r'([A-Za-z]+)\.', name)
    title = title_match.group(1) if title_match else 'Unknown'
    
    # Split by comma to separate surname from rest
    parts = name.split(',')
    surname = parts[0].strip() if parts else 'Unknown'
    
    # Extract first name, middle name, and other components
    if len(parts) > 1:
        remaining = parts[1].strip()
        
        # Remove title from remaining part
        remaining = re.sub(r'[A-Za-z]+\.\s*', '', remaining, count=1)
        
        # Extract nickname (in quotes)
        nickname_match = re.search(r'"([^"]+)"', remaining)
        nickname = nickname_match.group(1) if nickname_match else ''
        remaining = re.sub(r'"[^"]+"', '', remaining)
        
        # Extract maiden name (in parentheses)
        maiden_match = re.search(r'\(([^)]+)\)', remaining)
        maiden_name = maiden_match.group(1) if maiden_match else ''
        remaining = re.sub(r'\([^)]+\)', '', remaining)
        
        # Split remaining into first and middle names
        name_parts = remaining.strip().split()
        first_name = name_parts[0] if name_parts else 'Unknown'
        middle_name = ' '.join(name_parts[1:]) if len(name_parts) > 1 else ''
    else:
        first_name = middle_name = maiden_name = nickname = 'Unknown'
    
    return {
        'title': title,
        'surname': surname,
        'first_name': first_name,
        'middle_name': middle_name,
        'maiden_name': maiden_name,
        'nickname': nickname
    }

# Apply name extraction
name_components = data['Name'].apply(extract_name_components)
name_df = pd.DataFrame(name_components.tolist())

# Add to main dataset
for col in name_df.columns:
    data[f'name_{col}'] = name_df[col]

print("✅ Name components extracted")
print(f"Unique titles: {data['name_title'].nunique()}")
print(f"Unique surnames: {data['name_surname'].nunique()}")

# Display sample name parsing
sample_names = data[['Name', 'name_title', 'name_surname', 'name_first_name', 'name_nickname']].head(10)
display(sample_names)

### 1.2 Ethnic and Geographic Classification

In [ ]:
# Define ethnic/geographic patterns based on surname analysis
ethnic_patterns = {
    'Irish': ['O\'', 'Mc', 'Mac', 'Kelly', 'Murphy', 'Sullivan', 'Ryan', 'Connor', 'Flynn', 'Doyle'],
    'Scottish': ['Mac', 'Mc', 'Campbell', 'Stewart', 'Robertson', 'Thomson', 'Anderson', 'Wilson'],
    'English': ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Miller', 'Davis', 'Wilson', 'Moore', 'Taylor'],
    'German': ['Mueller', 'Schmidt', 'Schneider', 'Fischer', 'Weber', 'Meyer', 'Wagner', 'Becker', 'Schulz'],
    'Scandinavian': ['sen$', 'son$', 'berg$', 'ström$', 'lund$', 'borg$', 'dahl$', 'Olsen', 'Hansen', 'Nielsen'],
    'French': ['eau$', 'ier$', 'ais$', 'ois$', 'Dubois', 'Martin', 'Bernard', 'Durand', 'Moreau'],
    'Italian': ['ini$', 'ino$', 'etti$', 'elli$', 'Rossi', 'Ferrari', 'Esposito', 'Bianchi', 'Romano'],
    'Spanish': ['ez$', 'es$', 'Garcia', 'Rodriguez', 'Martinez', 'Hernandez', 'Lopez', 'Gonzalez'],
    'Eastern_European': ['ski$', 'sky$', 'ovic$', 'ovich$', 'enko$', 'Kowalski', 'Nowak', 'Wiśniewski'],
    'Jewish': ['stein$', 'berg$', 'man$', 'Cohen', 'Levy', 'Rosen', 'Gold', 'Silver'],
    'Arabic': ['Al ', 'El ', 'Ibn', 'Hassan', 'Ahmed', 'Mohammed', 'Abdullah'],
    'Chinese': ['Wong', 'Wang', 'Li', 'Chen', 'Liu', 'Yang', 'Huang', 'Zhao', 'Wu'],
    'Japanese': ['Yamamoto', 'Tanaka', 'Watanabe', 'Ito', 'Nakamura', 'Kobayashi', 'Kato', 'Yoshida']
}

def classify_ethnicity(surname):
    """
    Classify ethnicity based on surname patterns
    """
    if pd.isna(surname) or surname == 'Unknown':
        return 'Unknown'
    
    surname = surname.strip()
    
    for ethnicity, patterns in ethnic_patterns.items():
        for pattern in patterns:
            if pattern.endswith('$'):
                # Regex pattern
                if re.search(pattern, surname, re.IGNORECASE):
                    return ethnicity
            else:
                # Exact match or substring
                if pattern.lower() in surname.lower():
                    return ethnicity
    
    return 'Other_European'  # Default classification

# Apply ethnicity classification
data['ethnicity'] = data['name_surname'].apply(classify_ethnicity)

# Create ethnicity distribution
ethnicity_survival = data.groupby('ethnicity').agg({
    'Survived': ['count', 'sum', 'mean']
}).round(3)
ethnicity_survival.columns = ['Count', 'Survivors', 'Survival_Rate']
ethnicity_survival = ethnicity_survival.sort_values('Count', ascending=False)

print("🌍 Ethnicity Classification Results:")
display(ethnicity_survival)

# Visualize ethnicity distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Count by ethnicity
ethnicity_survival['Count'].plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Passenger Count by Ethnicity')
ax1.set_ylabel('Count')
ax1.tick_params(axis='x', rotation=45)

# Survival rate by ethnicity
ethnicity_survival['Survival_Rate'].plot(kind='bar', ax=ax2, color='coral')
ax2.set_title('Survival Rate by Ethnicity')
ax2.set_ylabel('Survival Rate')
ax2.tick_params(axis='x', rotation=45)
ax2.axhline(y=data['Survived'].mean(), color='red', linestyle='--', alpha=0.7, label='Overall Average')
ax2.legend()

plt.tight_layout()
plt.show()

### 1.3 Name Length and Complexity Analysis

In [ ]:
# Name complexity features
data['name_length'] = data['Name'].str.len()
data['name_word_count'] = data['Name'].str.split().str.len()
data['has_nickname'] = (data['name_nickname'] != '').astype(int)
data['has_maiden_name'] = (data['name_maiden_name'] != '').astype(int)
data['has_middle_name'] = (data['name_middle_name'] != '').astype(int)
data['name_complexity_score'] = (data['has_nickname'] + data['has_maiden_name'] + 
                                 data['has_middle_name'] + (data['name_word_count'] > 4).astype(int))

# Title frequency and rarity
title_counts = data['name_title'].value_counts()
data['title_frequency'] = data['name_title'].map(title_counts)
data['title_rarity'] = 1 / data['title_frequency']  # Inverse frequency

# Surname frequency (family connections)
surname_counts = data['name_surname'].value_counts()
data['surname_frequency'] = data['name_surname'].map(surname_counts)
data['is_unique_surname'] = (data['surname_frequency'] == 1).astype(int)
data['has_family_aboard'] = (data['surname_frequency'] > 1).astype(int)

# Statistical analysis of name features
name_features = ['name_length', 'name_word_count', 'name_complexity_score', 
                 'title_rarity', 'surname_frequency']

correlation_matrix = data[name_features + ['Survived']].corr()

print("📊 Name Complexity Analysis:")
for feature in name_features:
    corr_with_survival = correlation_matrix.loc[feature, 'Survived']
    print(f"{feature}: {corr_with_survival:.3f} correlation with survival")

# Visualize name complexity vs survival
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, feature in enumerate(name_features[:4]):
    data.boxplot(column=feature, by='Survived', ax=axes[i])
    axes[i].set_title(f'{feature} by Survival Status')
    axes[i].set_xlabel('Survived')

plt.suptitle('Name Feature Analysis', y=1.02)
plt.tight_layout()
plt.show()

# 🏗️ Section 2: Spatial Analysis of Cabin Positions
## Survival Corridors and Proximity Effects

### 2.1 Cabin Parsing and Spatial Mapping

In [ ]:
def parse_cabin_info(cabin):
    """
    Parse cabin information to extract deck, section, and position
    """
    if pd.isna(cabin) or cabin == '':
        return {
            'deck': 'Unknown',
            'cabin_number': -1,
            'cabin_position': 'Unknown',
            'multiple_cabins': 0,
            'cabin_count': 0
        }
    
    # Handle multiple cabins (separated by space)
    cabins = cabin.split()
    multiple_cabins = len(cabins) > 1
    
    # Parse first/primary cabin
    primary_cabin = cabins[0]
    
    # Extract deck (letter) and number
    match = re.match(r'([A-G])([0-9]+)', primary_cabin)
    if match:
        deck = match.group(1)
        cabin_number = int(match.group(2))
        
        # Determine position on deck (bow to stern approximation)
        if cabin_number <= 30:
            position = 'Forward'
        elif cabin_number <= 100:
            position = 'Midship'
        else:
            position = 'Aft'
    else:
        deck = 'Unknown'
        cabin_number = -1
        position = 'Unknown'
    
    return {
        'deck': deck,
        'cabin_number': cabin_number,
        'cabin_position': position,
        'multiple_cabins': int(multiple_cabins),
        'cabin_count': len(cabins)
    }

# Apply cabin parsing
cabin_info = data['Cabin'].apply(parse_cabin_info)
cabin_df = pd.DataFrame(cabin_info.tolist())

for col in cabin_df.columns:
    data[f'cabin_{col}'] = cabin_df[col]

# Deck hierarchy mapping (higher = more survival advantage)
deck_hierarchy = {
    'A': 6, 'B': 5, 'C': 4, 'D': 3, 'E': 2, 'F': 1, 'G': 0, 'Unknown': -1
}
data['deck_level'] = data['cabin_deck'].map(deck_hierarchy)

print("🏗️ Cabin Analysis Results:")
print(f"Passengers with cabin data: {(data['cabin_deck'] != 'Unknown').sum()} ({(data['cabin_deck'] != 'Unknown').mean():.1%})")
print(f"Passengers with multiple cabins: {data['cabin_multiple_cabins'].sum()}")

# Deck survival analysis
deck_survival = data[data['cabin_deck'] != 'Unknown'].groupby('cabin_deck').agg({
    'Survived': ['count', 'sum', 'mean'],
    'Pclass': 'mean'
}).round(3)
deck_survival.columns = ['Count', 'Survivors', 'Survival_Rate', 'Avg_Class']

display(deck_survival)

# Visualize deck analysis
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

# Deck distribution
deck_counts = data[data['cabin_deck'] != 'Unknown']['cabin_deck'].value_counts().sort_index()
deck_counts.plot(kind='bar', ax=ax1, color='lightblue')
ax1.set_title('Passengers by Deck')
ax1.set_ylabel('Count')

# Survival by deck
deck_survival['Survival_Rate'].plot(kind='bar', ax=ax2, color='coral')
ax2.set_title('Survival Rate by Deck')
ax2.set_ylabel('Survival Rate')
ax2.axhline(y=data['Survived'].mean(), color='red', linestyle='--', alpha=0.7)

# Position analysis
position_survival = data[data['cabin_position'] != 'Unknown'].groupby('cabin_position')['Survived'].mean()
position_survival.plot(kind='bar', ax=ax3, color='lightgreen')
ax3.set_title('Survival Rate by Cabin Position')
ax3.set_ylabel('Survival Rate')
ax3.axhline(y=data['Survived'].mean(), color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

### 2.2 Proximity and Clustering Analysis

In [ ]:
# Create cabin proximity features
def calculate_cabin_proximity_features(data):
    """
    Calculate features based on cabin proximity and clustering
    """
    # Filter passengers with known cabins
    known_cabins = data[data['cabin_cabin_number'] != -1].copy()
    
    proximity_features = []
    
    for idx, passenger in known_cabins.iterrows():
        deck = passenger['cabin_deck']
        cabin_num = passenger['cabin_cabin_number']
        
        # Find nearby passengers (same deck, within 10 cabin numbers)
        nearby_mask = (
            (known_cabins['cabin_deck'] == deck) & 
            (abs(known_cabins['cabin_cabin_number'] - cabin_num) <= 10) &
            (known_cabins.index != idx)
        )
        
        nearby_passengers = known_cabins[nearby_mask]
        
        # Calculate proximity features
        features = {
            'nearby_passenger_count': len(nearby_passengers),
            'nearby_survivors': nearby_passengers['Survived'].sum(),
            'nearby_survival_rate': nearby_passengers['Survived'].mean() if len(nearby_passengers) > 0 else np.nan,
            'nearby_avg_age': nearby_passengers['Age'].mean() if len(nearby_passengers) > 0 else np.nan,
            'nearby_avg_fare': nearby_passengers['Fare'].mean() if len(nearby_passengers) > 0 else np.nan,
            'nearby_families': nearby_passengers[(nearby_passengers['SibSp'] > 0) | (nearby_passengers['Parch'] > 0)].shape[0]
        }
        
        proximity_features.append(features)
    
    return pd.DataFrame(proximity_features, index=known_cabins.index)

# Calculate proximity features
proximity_df = calculate_cabin_proximity_features(data)

# Merge back to main dataset
for col in proximity_df.columns:
    data[col] = proximity_df[col]
    data[col] = data[col].fillna(-1)  # Fill NaN for passengers without cabin data

# Cabin density features
deck_passenger_counts = data[data['cabin_deck'] != 'Unknown']['cabin_deck'].value_counts()
data['deck_density'] = data['cabin_deck'].map(deck_passenger_counts).fillna(0)

# High-value cabin indicator (multiple cabins or premium location)
data['premium_cabin'] = (
    (data['cabin_multiple_cabins'] == 1) | 
    (data['deck_level'] >= 5) |
    (data['cabin_cabin_number'].between(1, 30))  # Forward cabins
).astype(int)

print("🎯 Proximity Analysis Results:")
proximity_cols = ['nearby_passenger_count', 'nearby_survival_rate', 'deck_density', 'premium_cabin']

for col in proximity_cols:
    if col in data.columns:
        valid_data = data[data[col] != -1] if col.startswith('nearby') else data
        if len(valid_data) > 0:
            corr = valid_data[col].corr(valid_data['Survived'])
            print(f"{col}: {corr:.3f} correlation with survival")

# Visualize proximity effects
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

# Premium cabin survival
premium_survival = data.groupby('premium_cabin')['Survived'].mean()
premium_survival.plot(kind='bar', ax=axes[0], color=['lightcoral', 'lightgreen'])
axes[0].set_title('Survival Rate: Premium vs Standard Cabins')
axes[0].set_xticklabels(['Standard', 'Premium'], rotation=0)

# Deck density vs survival
valid_density = data[data['deck_density'] > 0]
if len(valid_density) > 0:
    density_bins = pd.cut(valid_density['deck_density'], bins=3, labels=['Low', 'Medium', 'High'])
    density_survival = valid_density.groupby(density_bins)['Survived'].mean()
    density_survival.plot(kind='bar', ax=axes[1], color='skyblue')
    axes[1].set_title('Survival Rate by Deck Density')
    axes[1].tick_params(axis='x', rotation=45)

# Nearby survival rate effect
valid_nearby = data[data['nearby_survival_rate'] != -1]
if len(valid_nearby) > 0:
    axes[2].scatter(valid_nearby['nearby_survival_rate'], valid_nearby['Survived'], alpha=0.6)
    axes[2].set_xlabel('Nearby Passengers Survival Rate')
    axes[2].set_ylabel('Individual Survival')
    axes[2].set_title('Individual vs Nearby Survival Correlation')

# Multiple cabins effect
multi_cabin_survival = data.groupby('cabin_multiple_cabins')['Survived'].mean()
multi_cabin_survival.plot(kind='bar', ax=axes[3], color=['orange', 'purple'])
axes[3].set_title('Survival Rate: Single vs Multiple Cabins')
axes[3].set_xticklabels(['Single', 'Multiple'], rotation=0)

plt.tight_layout()
plt.show()

# 🤝 Section 3: Interaction Effects Between Social Class and Family Structure
## Advanced Statistical Modeling of Complex Relationships

### 3.1 Family Structure Analysis

In [ ]:
# Enhanced family structure features
data['family_size'] = data['SibSp'] + data['Parch'] + 1
data['is_alone'] = (data['family_size'] == 1).astype(int)
data['has_spouse'] = (data['SibSp'] > 0).astype(int)
data['has_children'] = (data['Parch'] > 0).astype(int)
data['has_parents'] = ((data['Parch'] > 0) & (data['Age'] < 18)).astype(int)

# Family type classification
def classify_family_type(row):
    if row['family_size'] == 1:
        return 'Solo'
    elif row['has_spouse'] and not row['has_children']:
        return 'Couple'
    elif row['has_children'] and row['Age'] > 18:
        return 'Parent'
    elif row['has_parents']:
        return 'Child'
    elif row['SibSp'] > 0 and row['Age'] < 18:
        return 'Sibling_Group'
    else:
        return 'Extended_Family'

data['family_type'] = data.apply(classify_family_type, axis=1)

# Family size categories
def categorize_family_size(size):
    if size == 1:
        return 'Solo'
    elif size <= 4:
        return 'Small_Family'
    elif size <= 6:
        return 'Medium_Family'
    else:
        return 'Large_Family'

data['family_size_category'] = data['family_size'].apply(categorize_family_size)

# Economic burden per family member
data['fare_per_person'] = data['Fare'] / data['family_size']
data['economic_burden'] = data['family_size'] / (data['Fare'] + 1)  # +1 to avoid division by zero

print("👨‍👩‍👧‍👦 Family Structure Analysis:")
family_survival = data.groupby('family_type').agg({
    'Survived': ['count', 'mean'],
    'Age': 'mean',
    'Fare': 'mean'
}).round(3)
family_survival.columns = ['Count', 'Survival_Rate', 'Avg_Age', 'Avg_Fare']
display(family_survival)

# Visualize family effects
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Family size vs survival
size_survival = data.groupby('family_size')['Survived'].mean()
size_survival.plot(kind='bar', ax=ax1, color='lightblue')
ax1.set_title('Survival Rate by Family Size')
ax1.set_xlabel('Family Size')
ax1.axhline(y=data['Survived'].mean(), color='red', linestyle='--', alpha=0.7)

# Family type vs survival
type_survival = data.groupby('family_type')['Survived'].mean()
type_survival.plot(kind='bar', ax=ax2, color='lightgreen')
ax2.set_title('Survival Rate by Family Type')
ax2.tick_params(axis='x', rotation=45)
ax2.axhline(y=data['Survived'].mean(), color='red', linestyle='--', alpha=0.7)

# Economic burden vs survival
data.boxplot(column='economic_burden', by='Survived', ax=ax3)
ax3.set_title('Economic Burden by Survival Status')
ax3.set_xlabel('Survived')

# Fare per person vs survival
data.boxplot(column='fare_per_person', by='Survived', ax=ax4)
ax4.set_title('Fare per Person by Survival Status')
ax4.set_xlabel('Survived')

plt.suptitle('Family Structure Analysis', y=1.02)
plt.tight_layout()
plt.show()

### 3.2 Class-Family Interaction Effects

In [ ]:
# Create interaction features
data['class_family_interaction'] = data['Pclass'] * data['family_size']
data['class_alone_interaction'] = data['Pclass'] * data['is_alone']
data['class_fare_interaction'] = data['Pclass'] * data['Fare']

# Advanced interaction analysis
class_family_crosstab = pd.crosstab(
    [data['Pclass'], data['family_size_category']], 
    data['Survived'], 
    normalize='index'
).round(3)

print("🎭 Class-Family Interaction Analysis:")
print("Survival rates by Class and Family Size:")
display(class_family_crosstab)

# Statistical significance testing
from scipy.stats import chi2_contingency

# Test independence of class, family size, and survival
contingency_table = pd.crosstab(
    [data['Pclass'], data['family_size_category']], 
    data['Survived']
)

chi2, p_value, dof, expected = chi2_contingency(contingency_table)
print(f"\n📊 Chi-square test results:")
print(f"Chi-square statistic: {chi2:.3f}")
print(f"P-value: {p_value:.6f}")
print(f"Degrees of freedom: {dof}")
print(f"Significant interaction: {'Yes' if p_value < 0.05 else 'No'}")

# Create polynomial features for complex interactions
interaction_features = ['Pclass', 'family_size', 'Age', 'Fare']
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly_features = poly.fit_transform(data[interaction_features].fillna(data[interaction_features].median()))
poly_feature_names = poly.get_feature_names_out(interaction_features)

# Add selected polynomial features
important_poly_features = [
    'Pclass family_size', 'Pclass Age', 'Pclass Fare', 'family_size Age', 'family_size Fare'
]

for i, feature_name in enumerate(poly_feature_names):
    if feature_name in important_poly_features:
        data[feature_name.replace(' ', '_')] = poly_features[:, i]

# Visualize interaction effects
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Heatmap of class-family survival rates
survival_pivot = data.pivot_table(
    values='Survived', 
    index='Pclass', 
    columns='family_size_category', 
    aggfunc='mean'
)
sns.heatmap(survival_pivot, annot=True, cmap='RdYlBu_r', ax=axes[0,0], cbar_kws={'label': 'Survival Rate'})
axes[0,0].set_title('Survival Rate: Class vs Family Size')

# Class-family size distribution
class_family_counts = data.groupby(['Pclass', 'family_size_category']).size().unstack(fill_value=0)
class_family_counts.plot(kind='bar', ax=axes[0,1], stacked=True)
axes[0,1].set_title('Passenger Distribution: Class vs Family Size')
axes[0,1].legend(title='Family Size')
axes[0,1].tick_params(axis='x', rotation=0)

# Interaction effect visualization
for pclass in [1, 2, 3]:
    class_data = data[data['Pclass'] == pclass]
    family_survival = class_data.groupby('family_size')['Survived'].mean()
    axes[1,0].plot(family_survival.index, family_survival.values, 
                   marker='o', label=f'Class {pclass}', linewidth=2)

axes[1,0].set_xlabel('Family Size')
axes[1,0].set_ylabel('Survival Rate')
axes[1,0].set_title('Survival Rate by Family Size and Class')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Economic interaction: Fare per person by class
data.boxplot(column='fare_per_person', by='Pclass', ax=axes[1,1])
axes[1,1].set_title('Fare per Person by Class')
axes[1,1].set_xlabel('Class')

plt.tight_layout()
plt.show()

# Feature importance analysis for interaction effects
interaction_cols = [col for col in data.columns if '_' in col and any(base in col for base in ['Pclass', 'family_size', 'Age', 'Fare'])]
print(f"\n🔍 Created {len(interaction_cols)} interaction features")

# 🎯 Section 4: Feature Selection and Validation
## Statistical Rigor in Feature Evaluation

### 4.1 Comprehensive Feature Selection

In [ ]:
# Prepare feature matrix
# Select all engineered features
feature_columns = [
    # Original features
    'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked',
    
    # Name-based features
    'name_length', 'name_word_count', 'name_complexity_score', 'title_rarity', 
    'surname_frequency', 'has_family_aboard', 'has_nickname', 'has_maiden_name',
    
    # Cabin-based features
    'deck_level', 'cabin_multiple_cabins', 'premium_cabin', 'deck_density',
    'nearby_passenger_count', 'nearby_survival_rate',
    
    # Family structure features
    'family_size', 'is_alone', 'has_spouse', 'has_children', 'fare_per_person', 'economic_burden',
    
    # Interaction features
    'class_family_interaction', 'class_alone_interaction'
]

# Create categorical encodings
feature_data = data.copy()

# Encode categorical variables
le_sex = LabelEncoder()
feature_data['Sex'] = le_sex.fit_transform(feature_data['Sex'])

le_embarked = LabelEncoder()
feature_data['Embarked'] = le_embarked.fit_transform(feature_data['Embarked'].fillna('S'))

le_ethnicity = LabelEncoder()
feature_data['ethnicity_encoded'] = le_ethnicity.fit_transform(feature_data['ethnicity'])
feature_columns.append('ethnicity_encoded')

# Handle missing values
for col in feature_columns:
    if col in feature_data.columns:
        if feature_data[col].dtype in ['object']:
            feature_data[col] = feature_data[col].fillna('Unknown')
        else:
            # For nearby features, -1 indicates no cabin data
            if 'nearby' in col:
                feature_data[col] = feature_data[col].fillna(-1)
            else:
                feature_data[col] = feature_data[col].fillna(feature_data[col].median())

# Create feature matrix
available_features = [col for col in feature_columns if col in feature_data.columns]
X = feature_data[available_features]
y = feature_data['Survived']

print(f"📊 Feature Selection Analysis")
print(f"Total features available: {len(available_features)}")
print(f"Dataset shape: {X.shape}")

# Remove any remaining non-numeric columns
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
X_numeric = X[numeric_features]

print(f"Numeric features for analysis: {len(numeric_features)}")

# Multiple feature selection methods
feature_scores = {}

# 1. Univariate feature selection (F-test)
f_selector = SelectKBest(score_func=f_classif, k='all')
f_selector.fit(X_numeric, y)
feature_scores['f_test'] = dict(zip(numeric_features, f_selector.scores_))

# 2. Mutual information
mi_scores = mutual_info_classif(X_numeric, y, random_state=42)
feature_scores['mutual_info'] = dict(zip(numeric_features, mi_scores))

# 3. Random Forest feature importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_numeric, y)
feature_scores['random_forest'] = dict(zip(numeric_features, rf.feature_importances_))

# 4. Correlation with target
correlations = X_numeric.corrwith(y).abs()
feature_scores['correlation'] = correlations.to_dict()

# Combine scores (normalized)
normalized_scores = {}
for method, scores in feature_scores.items():
    max_score = max(scores.values())
    normalized_scores[method] = {k: v/max_score for k, v in scores.items()}

# Calculate ensemble score
ensemble_scores = {}
for feature in numeric_features:
    scores = [normalized_scores[method][feature] for method in normalized_scores.keys()]
    ensemble_scores[feature] = np.mean(scores)

# Create comprehensive results DataFrame
results_df = pd.DataFrame({
    'Feature': numeric_features,
    'F_Test_Score': [feature_scores['f_test'][f] for f in numeric_features],
    'Mutual_Info': [feature_scores['mutual_info'][f] for f in numeric_features],
    'RF_Importance': [feature_scores['random_forest'][f] for f in numeric_features],
    'Correlation': [feature_scores['correlation'][f] for f in numeric_features],
    'Ensemble_Score': [ensemble_scores[f] for f in numeric_features]
}).sort_values('Ensemble_Score', ascending=False)

print("\n🏆 Top 15 Features by Ensemble Score:")
display(results_df.head(15))

# Visualize feature importance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Top features by different methods
top_n = 10

# F-test scores
top_f_test = results_df.nlargest(top_n, 'F_Test_Score')
axes[0,0].barh(range(len(top_f_test)), top_f_test['F_Test_Score'], color='lightblue')
axes[0,0].set_yticks(range(len(top_f_test)))
axes[0,0].set_yticklabels(top_f_test['Feature'], fontsize=8)
axes[0,0].set_title('Top Features: F-Test Scores')
axes[0,0].invert_yaxis()

# Random Forest importance
top_rf = results_df.nlargest(top_n, 'RF_Importance')
axes[0,1].barh(range(len(top_rf)), top_rf['RF_Importance'], color='lightgreen')
axes[0,1].set_yticks(range(len(top_rf)))
axes[0,1].set_yticklabels(top_rf['Feature'], fontsize=8)
axes[0,1].set_title('Top Features: Random Forest Importance')
axes[0,1].invert_yaxis()

# Mutual Information
top_mi = results_df.nlargest(top_n, 'Mutual_Info')
axes[1,0].barh(range(len(top_mi)), top_mi['Mutual_Info'], color='coral')
axes[1,0].set_yticks(range(len(top_mi)))
axes[1,0].set_yticklabels(top_mi['Feature'], fontsize=8)
axes[1,0].set_title('Top Features: Mutual Information')
axes[1,0].invert_yaxis()

# Ensemble scores
top_ensemble = results_df.nlargest(top_n, 'Ensemble_Score')
axes[1,1].barh(range(len(top_ensemble)), top_ensemble['Ensemble_Score'], color='gold')
axes[1,1].set_yticks(range(len(top_ensemble)))
axes[1,1].set_yticklabels(top_ensemble['Feature'], fontsize=8)
axes[1,1].set_title('Top Features: Ensemble Score')
axes[1,1].invert_yaxis()

plt.tight_layout()
plt.show()

### 4.2 Feature Performance Validation

In [ ]:
# Progressive feature addition analysis
def evaluate_feature_sets(X, y, feature_rankings, step_size=5):
    """
    Evaluate model performance with progressively more features
    """
    results = []
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    for i in range(step_size, len(feature_rankings) + 1, step_size):
        # Select top i features
        selected_features = feature_rankings[:i]
        X_selected = X[selected_features]
        
        # Cross-validation
        cv_scores = cross_val_score(rf_model, X_selected, y, cv=5, scoring='accuracy')
        
        results.append({
            'num_features': i,
            'features': selected_features,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std()
        })
    
    return results

# Get feature rankings from ensemble scores
top_features = results_df.sort_values('Ensemble_Score', ascending=False)['Feature'].tolist()

# Evaluate different feature set sizes
performance_results = evaluate_feature_sets(X_numeric, y, top_features, step_size=3)

# Convert to DataFrame for analysis
performance_df = pd.DataFrame(performance_results)

print("🎯 Feature Set Performance Analysis:")
for _, row in performance_df.iterrows():
    print(f"{row['num_features']:2d} features: {row['cv_mean']:.4f} ± {row['cv_std']:.4f}")

# Find optimal number of features
best_performance = performance_df.loc[performance_df['cv_mean'].idxmax()]
print(f"\n🏆 Best performance: {best_performance['cv_mean']:.4f} with {best_performance['num_features']} features")

# Visualize performance curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Performance vs number of features
ax1.plot(performance_df['num_features'], performance_df['cv_mean'], marker='o', linewidth=2, markersize=8)
ax1.fill_between(performance_df['num_features'], 
                 performance_df['cv_mean'] - performance_df['cv_std'],
                 performance_df['cv_mean'] + performance_df['cv_std'], 
                 alpha=0.2)
ax1.axvline(x=best_performance['num_features'], color='red', linestyle='--', 
            label=f'Optimal: {best_performance["num_features"]} features')
ax1.set_xlabel('Number of Features')
ax1.set_ylabel('Cross-Validation Accuracy')
ax1.set_title('Model Performance vs Feature Count')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Feature importance heatmap for top features
top_15_features = top_features[:15]
importance_matrix = np.array([
    [normalized_scores['f_test'][f] for f in top_15_features],
    [normalized_scores['mutual_info'][f] for f in top_15_features],
    [normalized_scores['random_forest'][f] for f in top_15_features],
    [normalized_scores['correlation'][f] for f in top_15_features]
])

sns.heatmap(importance_matrix, 
            xticklabels=[f[:15] + '...' if len(f) > 15 else f for f in top_15_features],
            yticklabels=['F-Test', 'Mutual Info', 'Random Forest', 'Correlation'],
            annot=True, fmt='.3f', cmap='YlOrRd', ax=ax2)
ax2.set_title('Feature Importance Heatmap (Top 15 Features)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Final feature recommendations
optimal_features = top_features[:int(best_performance['num_features'])]

print(f"\n🎯 Recommended Feature Set ({len(optimal_features)} features):")
for i, feature in enumerate(optimal_features, 1):
    ensemble_score = ensemble_scores[feature]
    print(f"{i:2d}. {feature:<25} (Score: {ensemble_score:.4f})")

# Category breakdown of selected features
feature_categories = {
    'Original': ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'],
    'Name_NLP': [f for f in optimal_features if f.startswith('name_') or f in ['title_rarity', 'surname_frequency', 'has_family_aboard', 'has_nickname', 'has_maiden_name', 'ethnicity_encoded']],
    'Cabin_Spatial': [f for f in optimal_features if f.startswith('cabin_') or f.startswith('deck_') or f.startswith('nearby_') or f == 'premium_cabin'],
    'Family_Structure': [f for f in optimal_features if 'family' in f or f in ['is_alone', 'has_spouse', 'has_children', 'fare_per_person', 'economic_burden']],
    'Interaction': [f for f in optimal_features if 'interaction' in f or '_' in f and any(base in f for base in ['Pclass', 'class'])]
}

print("\n📊 Feature Category Breakdown:")
for category, features in feature_categories.items():
    selected_in_category = [f for f in features if f in optimal_features]
    if selected_in_category:
        print(f"{category}: {len(selected_in_category)} features - {', '.join(selected_in_category[:3])}{'...' if len(selected_in_category) > 3 else ''}")

## 📈 Summary and Business Impact

This advanced feature engineering analysis demonstrates sophisticated data science techniques that significantly enhance predictive power:

### Key Achievements:

1. **NLP Analysis**: Extracted ethnic and cultural patterns from passenger names, revealing survival disparities across different communities

2. **Spatial Analysis**: Analyzed cabin positions and proximity effects, identifying survival corridors and the impact of premium locations

3. **Interaction Modeling**: Discovered complex relationships between social class and family dynamics that traditional analysis would miss

4. **Statistical Rigor**: Applied multiple feature selection techniques with cross-validation to ensure robust feature identification

### Business Applications:
- **Maritime Safety**: Optimize evacuation procedures based on spatial analysis
- **Insurance Risk Assessment**: Incorporate demographic and social factors in risk models
- **Emergency Planning**: Understand how family structures affect emergency response
- **Historical Analysis**: Quantify social inequality impacts in disaster scenarios

### Technical Skills Demonstrated:
- Advanced feature engineering with domain expertise
- NLP techniques for categorical data enrichment
- Spatial data analysis and geographic information processing
- Statistical interaction modeling and hypothesis testing
- Comprehensive feature selection and validation methodologies

This notebook showcases the type of sophisticated analysis that distinguishes senior data scientists in their ability to extract hidden insights from complex datasets.